<a href="https://colab.research.google.com/github/cyamize/dacon-winning-data/blob/main/%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%A0%84%EC%B2%98%EB%A6%AC_kbo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터 전처리
예측 모델 구축 시 튜닝도 중요하지만 양질의 데이터로 구성된 학습데이터가 우선적으로 필요합니다. 모델은 학습데이터를 바탕으로 구축되는데, 학습데이터의 질이 나쁘면 모델의 성능이 나쁜 것이 일반적이다.


---
가설을 세우고, 가설에 문제없음을 밝힘으로써 데이터 분석을 전개한다.

- 가설 : KBO에서 우수한 활약을 보인 투수들은 MLB에서 제구력이 뛰어날 것이다

참 거짓을 판별하기 위해서 다음을 결정해야한다.

1. 가설 확인의 대상이 되는 투수는?
2. KBO에서 우수한 활약은 어떻게 측정할 것인가?
3. MLB에서 제구력은 어떻게 측정할 것인가?

이 가설은 MLB에서 투구 기록과 KBO에서의 투구 기록이 모두 기록된 투수에 대해서만 확인이 가능하다. 해당 조건에 부합하는 선수를 추출한다.


다음 코드를 통해 제공된 데이터를 불러온다

In [ ]:
import pandas as pd

# 데이터셋 부르기
atKbo_11_18_KboRegSsn = pd.read_csv('data/kbo_yearly_foreigners_2011_2018.csv')
atKbo_11_18_MlbTot = pd.read_csv('data/fangraphs_foreigners_2011_2018.csv')
atKbo_11_18_StatCast = pd.read_csv('data/baseball_savant_foreigners_2011_2018.csv')
atKbo_19_MlbTot = pd.read_csv('data/fangraphs_foreigners_2019.csv')
atKbo_19_StatCast = pd.read_csv('data/baseball_savant_foreigners_2019.csv')

대회에서 제공된 데이터는 pandas 패키지의 read_csv()함수를 통해 CSV 함수를 읽을 수 있다.

각 변수별로 저장된 파일은 다음과 같다.
- atKbo_11_18_KboRegSsn : 2011년부터 2018년까지 KBO 정규시즌 성적
- atKbo_11_18_MlbTot : KBO 입성 전, MLB에서의 2011년부터 2018년까지 정규시즌 성적
- atKbo_11_18_StatCast : KBO 입성 전, 2011년부터 2018년까지 MLB에서 스탯캐스트 데이터
- atKbo_19_MlbTot : 2019년 MLB에서 역대 정규시즌 성적
- atKbo_19_StatCast : 2019년 MLB에서 스탯캐스트 데이터

pd.DataFrame.head() 함수를 통해 상위 10개 행의 KBO 정규시즌 성적 확인

In [ ]:
atKbo_11_18_KboRegSsn.head(10)

,pitcher_name,year,team,ERA,TBF,H,HR,BB,HBP,SO,year_born
0,니퍼트,2011,두산,2.55,763,150,8,64,10,150,NaN
1,니퍼트,2012,두산,3.20,785,156,15,68,8,126,NaN
2,니퍼트,2013,두산,3.58,482,108,7,34,4,104,NaN
3,니퍼트,2014,두산,3.81,760,186,17,48,6,158,NaN
4,니퍼트,2015,두산,5.10,404,104,4,33,4,76,NaN
5,니퍼트,2016,두산,2.95,701,151,15,57,9,142,NaN
6,니퍼트,2017,두산,4.06,782,175,20,77,10,161,NaN
7,니퍼트,2018,KT,4.25,765,209,26,39,9,165,NaN
8,다이아몬드,2017,SK,4.42,581,163,11,35,9,59,NaN
9,듀브론트,2018,롯데,4.92,629,162,13,62,8,109,NaN


KBO 기록에 팀 소속과 더불어 연도별 ERA, TBF, H, HR 등의 지표가 기록됨을 확인

MLB 정규시즌 성적 확인

In [ ]:
atKbo_11_18_MlbTot.head(10)

,pitcher_name,year,ERA,WAR,TBF,H,HR,BB,HBP,SO,WHIP,BABIP,FIP,LD%,GB%,FB%,IFFB%,SwStr%,Swing%
0,오간도,2011.0,3.51,3.3,693.0,149.0,16.0,43.0,7.0,126.0,1.14,0.265,3.65,0.237,0.364,0.674,0.147,0.090,0.475
1,험버,2011.0,3.75,3.2,676.0,151.0,14.0,41.0,6.0,116.0,1.18,0.275,3.58,0.168,0.471,0.458,0.094,0.092,0.463
2,루카스,2012.0,3.76,2.8,827.0,185.0,13.0,78.0,1.0,140.0,1.36,0.289,3.75,0.203,0.572,0.707,0.082,0.062,0.424
3,다이아몬드,2012.0,3.54,2.2,714.0,184.0,17.0,31.0,4.0,90.0,1.24,0.292,3.94,0.210,0.534,0.597,0.040,0.068,0.467
4,듀브론트,2013.0,4.32,2.2,705.0,161.0,13.0,71.0,5.0,139.0,1.43,0.310,3.78,0.199,0.456,0.633,0.127,0.077,0.434
5,스와잭,2017.0,2.33,2.2,303.0,58.0,6.0,22.0,2.0,91.0,1.03,0.286,2.74,0.155,0.439,0.485,0.145,0.142,0.500
6,듀브론트,2012.0,4.86,1.3,709.0,162.0,24.0,71.0,5.0,167.0,1.45,0.312,4.37,0.234,0.437,0.635,0.086,0.096,0.442
7,탈보트,2010.0,4.41,1.2,696.0,169.0,13.0,69.0,8.0,88.0,1.49,0.301,4.48,0.169,0.478,0.495,0.070,0.060,0.428
8,비야누에바,2011.0,4.04,1.2,454.0,103.0,11.0,32.0,4.0,68.0,1.26,0.271,4.10,0.219,0.356,0.462,0.151,0.078,0.433
9,비야누에바,2014.0,4.64,1.2,343.0,89.0,6.0,19.0,3.0,72.0,1.39,0.342,3.13,0.202,0.413,0.474,0.161,0.113,0.486


MLB 기록에는 연도별 ERA, TBF, H, HR 등의 지표가 기록


스탯캐스트에 기록된 값 확인

In [ ]:
atKbo_11_18_StatCast.head(10)

,game_date,release_speed,batter,pitcher,events,description,zone,stand,p_throws,bb_type,balls,strikes,pfx_x,pfx_z,plate_x,plate_z,ax,ay,az,launch_speed,launch_angle,release_spin_rate,pitch_name,pitcher_name
0,2010-10-09,93.0,425834.0,430962,field_out,hit_into_play,4.0,R,R,popup,0,0,-0.604558,1.912833,-0.626,2.771,-6.404,26.077,-9.694,NaN,NaN,NaN,4-Seam Fastball,니퍼트
1,2010-10-09,95.2,150289.0,430962,home_run,hit_into_play_score,5.0,L,R,fly_ball,3,2,-0.857842,1.909967,-0.112,2.401,-10.168,28.786,-8.895,NaN,NaN,NaN,4-Seam Fastball,니퍼트
2,2010-10-09,94.3,150289.0,430962,NaN,foul,2.0,L,R,NaN,3,1,-0.536367,2.014600,0.272,3.052,-5.605,26.657,-7.794,NaN,NaN,NaN,4-Seam Fastball,니퍼트
3,2010-10-09,94.0,150289.0,430962,NaN,blocked_ball,13.0,L,R,NaN,2,1,-0.806350,2.167967,-1.193,0.632,-9.099,30.273,-6.371,NaN,NaN,NaN,4-Seam Fastball,니퍼트
4,2010-10-09,92.8,150289.0,430962,NaN,ball,14.0,L,R,NaN,1,1,-0.571158,1.818233,0.370,1.383,-5.846,27.290,-11.262,NaN,NaN,NaN,4-Seam Fastball,니퍼트
5,2010-10-09,80.8,150289.0,430962,NaN,foul_tip,13.0,L,R,NaN,1,0,-1.197408,1.391100,-1.233,2.059,-10.897,18.539,-20.286,NaN,NaN,NaN,Changeup,니퍼트
6,2010-10-09,92.0,150289.0,430962,NaN,ball,9.0,L,R,NaN,0,0,-0.610125,1.947233,0.510,1.722,-6.251,27.786,-10.031,NaN,NaN,NaN,4-Seam Fastball,니퍼트
7,2010-10-09,92.5,430632.0,430962,field_out,hit_into_play,13.0,L,R,line_drive,2,0,-0.753467,2.040400,-1.010,2.212,-8.279,26.466,-8.376,NaN,NaN,NaN,4-Seam Fastball,니퍼트
8,2010-10-09,91.2,430632.0,430962,NaN,ball,13.0,L,R,NaN,1,0,-0.848100,1.683500,-0.106,1.418,-9.239,25.490,-13.556,NaN,NaN,NaN,4-Seam Fastball,니퍼트
9,2010-10-09,91.8,430632.0,430962,NaN,ball,11.0,L,R,NaN,0,0,-1.049892,1.521533,-1.426,2.971,-12.090,25.659,-15.250,NaN,NaN,NaN,2-Seam Fastball,니퍼트


스탯캐스트 데이터에는 투구별 속성값이 기록되어져 있다.

세 데이터의 공통 데이터로 투수 이름 정보인 pitcher_name 칼럼이 기록되어 있다. 이 때, 2011년부터 2018년 사이에 기록된 데이터라 투수 이름이 중복되어 있다. 데이터별로 고유 투수 이름을 확인하기 위해 판다스에 제공되는 unique()함수를 사용하여 투수가 몇 명 있는지 확인한다.

In [ ]:
# unique() 함수는 고유 데이터 값을 뽑아낼 때 사용
print('KBO: ', len(atKbo_11_18_KboRegSsn['pitcher_name'].unique()))  #역대 KBO 정규시즌 성적 데이터 내 투수이름
print('MLB: ', len(atKbo_11_18_MlbTot['pitcher_name'].unique()))  # KBO 입성 전 MLB 정규시즌 성적 데이터 내투수이름
print('StatCast', len(atKbo_11_18_StatCast['pitcher_name'].unique()))  # KBO 입성 전, MLB 스탯캐스트 데이터 내 투수이름

KBO:  62
MLB:  60
StatCast 60


KBO, MLB, 스탯캐스트에는 각각 다른 투수가 기록되어 있다. 앞서 세운 가설 확인을 위해서는 MLB와 KBO 모두 기록된 선수가 필요하다. 
- 3개의 데이터 집단 모두에서 기록이 존재하는 선수를 추출

set() 함수를 이용하여 데이터 타입을 set으로 변환 시킨 뒤, & 연산자를 이용하여 데이터 내 교집합을 추출한다. 

In [ ]:
target = (set(atKbo_11_18_KboRegSsn['pitcher_name'])&
          set(atKbo_11_18_MlbTot['pitcher_name'])&
          set(atKbo_11_18_StatCast['pitcher_name']))

print(type(target))

print(len(target))

<class 'set'>
57


위 코드를 통해 KBO, MLB, 스탯캐스트 데이터에 기록이 존재하는 타깃 집단이 추출되었다.


---


해당되는 투수의 KBO의 투수능력 측정은 KBO 첫 번째 시즌의 평균자책점을 통해 
평가한다. 이는 외국인 투수의 경우 팀 성적에 즉시 기여할 수 있는 선수를 찾는 것이 목적이기 때문이다.

KBO 데이터 내에서 타깃 집단의 투수들을 query()함수를 통해 해당 투수들의 KBO 첫 번째 시즌을 추출합니다. 

query() 함수는 다음과 같은 기능을 포함한다.
- 비교 연산자( ==, >, >=, <, <=, != )
- in 연산자( in, ==, not in, != )
- 논리 연산자(and, or, not)
- 외부 변수(또는 함수) 참조 연산
- 인덱스 검색
- 문자열 부분검색( str.contains, str.startswith, str.endswith )

query()함수 안에는 문자열이 입력되야 한다. 문자열로는 환경에 저장된 변수에 접근이 불가능하나, 문자열 내 특수문자@을 사용하면 현재 환경에 있는 target 변수에 접근 가능하다.

In [ ]:
# 최소값을 지니는 인덱스 출력
# 외부 변수 참조 연산 @를 사용하여 target 변수에 접근
_idx = atKbo_11_18_KboRegSsn.query('pitcher_name in @target').groupby('pitcher_name')['year'].idxmin().values 
firstYearInKBO_11_18 = atKbo_11_18_KboRegSsn.loc[_idx,:] # 행: _idx, 열: 전부
firstYearInKBO_11_18.head()

,pitcher_name,year,team,ERA,TBF,H,HR,BB,HBP,SO,year_born
0,니퍼트,2011,두산,2.55,763,150,8,64,10,150,NaN
8,다이아몬드,2017,SK,4.42,581,163,11,35,9,59,NaN
9,듀브론트,2018,롯데,4.92,629,162,13,62,8,109,NaN
10,레나도,2017,삼성,6.80,243,70,11,27,6,28,NaN
11,레온,2016,삼성,11.25,40,16,2,1,1,7,NaN


총 57명의 투수의 KBO의 성적을 추출

---
하지만 모든 투수의 기록이 유효하지 않다. 신뢰성 있는 지표는 충분한 양의 시행횟수가 필요합니다.

 여기서 시행횟수를 '투수가 상대한 타자의 수(Total Batters Faced, TBF)라 한다.


투수의 활약상을 평가할 때 대표적으로 쓰이는 지표인 평균자책점(ERA)은 낮을 수록 좋다.

TBF의 중앙값 이상, ERA의 중앙값 이하인 투수를 우수한 투수라 가정하고 코드를 작성


In [ ]:
TBF_median = firstYearInKBO_11_18['TBF'].median() # KBO에서 TBF 중앙값
ERA_median = firstYearInKBO_11_18['ERA'].median() # KOB에서 ERA 중앙값
Elite_11_18 = firstYearInKBO_11_18.query('TBF >=@ @TBF_median & ERA <= @ ERA_median')
Elite_11_18

,pitcher_name,year,team,ERA,TBF,H,HR,BB,HBP,SO,year_born
0,니퍼트,2011,두산,2.55,763,150,8,64,10,150,NaN
8,다이아몬드,2017,SK,4.42,581,163,11,35,9,59,NaN
12,레이예스,2013,SK,4.84,751,157,15,91,7,135,NaN
14,레일리,2015,롯데,3.91,768,182,20,57,15,134,1988년 06월 29일
25,린드블럼,2015,롯데,3.56,861,196,28,52,12,180,1987년 06월 15일
37,보우덴,2016,두산,3.80,742,159,17,54,6,160,NaN
43,샘슨,2018,한화,4.68,703,144,17,79,5,195,NaN
45,세든,2013,SK,2.98,780,169,14,73,11,160,NaN
48,소사,2012,KIA,3.54,614,137,9,39,8,104,NaN
67,웨버,2014,NC,4.58,512,132,13,42,3,85,NaN


In [ ]:
len(Elite_11_18)

18

해당 과정을 통해 총 18명의 성적이 우수한 투수의 유효한 투수 기록을 추출하였다.